In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

# 1. Read the data

In [2]:
#Lấy luôn dữ liệu đã loc của tác giả
test_dataset = pd.read_csv("../data_for_modeling/HDAC3_test.csv")
train_dataset = pd.read_csv("../data_for_modeling/HDAC3_train.csv")
dataset = pd.concat([train_dataset, test_dataset], axis=0)
dataset = dataset.reset_index()
print(len(train_dataset), len(test_dataset), len(dataset))

790 198 988


In [3]:
print(dataset.columns)
dataset.head()

Index(['index', 'ID', 'SMILES', 'IC50_mean', 'Type'], dtype='object')


,index,ID,SMILES,IC50_mean,Type
0,0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.1290,1
1,1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.4600,1
2,2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.0055,1
3,3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.2140,1
4,4,CHEMBL178456,O=C(CS)NCCCCCC(=O)Nc1cccc2cccnc12,1.3200,0


In [4]:
def check_activity_distribution(dataset, col_name):
    active_rows = dataset.loc[dataset[col_name] == 1]
    inactive_rows = dataset.loc[dataset[col_name] == 0]

    dataset_length = len(dataset)

    print("Total dataset")
    table = [['', 'Active', 'Inactive'], 
            ['Number', len(active_rows), len(inactive_rows)],
            ['Percentage (%)', len(active_rows)/dataset_length*100, len(inactive_rows)/dataset_length*100]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [5]:
check_activity_distribution(dataset=dataset, col_name="Type")

Total dataset
╒════════════════╤══════════╤════════════╕
│                │   Active │   Inactive │
╞════════════════╪══════════╪════════════╡
│ Number         │ 672      │   316      │
├────────────────┼──────────┼────────────┤
│ Percentage (%) │  68.0162 │    31.9838 │
╘════════════════╧══════════╧════════════╛


# 2. Labels Errors

## 2.1. Preparing data

In [6]:
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#Encoding labels
dataset_c = dataset.copy()
dataset_c.head()

,index,ID,SMILES,IC50_mean,Type
0,0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.1290,1
1,1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.4600,1
2,2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.0055,1
3,3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.2140,1
4,4,CHEMBL178456,O=C(CS)NCCCCCC(=O)Nc1cccc2cccnc12,1.3200,0


**We will do this in the MACCS keys**

In [7]:
def maccs_fpts(data):
    Maccs_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts = MACCSkeys.GenMACCSKeys(mol)
        mfpts = np.array(fpts)
        Maccs_fpts.append(mfpts)
    return np.array(Maccs_fpts)

In [8]:
smiles = dataset.SMILES
data = maccs_fpts(smiles)
data = pd.DataFrame(data=data)
labels = dataset['Type']
print(data.shape, labels.shape)

(988, 167) (988,)


## 2.2. Getting out-of-sample predicted probabilities

In [9]:
model = XGBClassifier(tree_method="hist", enable_categorical=True)
pred_probs = cross_val_predict(model, data, labels, method='predict_proba')
print(len(pred_probs))
print(pred_probs)

988
[[1.9454533e-01 8.0545467e-01]
 [2.6005447e-02 9.7399455e-01]
 [2.7531385e-04 9.9972469e-01]
 ...
 [8.2197452e-01 1.7802550e-01]
 [9.8611861e-01 1.3881392e-02]
 [4.8068762e-03 9.9519312e-01]]


### 3.2. Checking model accuracy on original data

Now that we have out-of-sample predicted probabilities, we can also check the model's (cross-val) accuracy on the original (noisy) data, so we'll have a baseline to compare our final results.

In [10]:
preds = np.argmax(pred_probs, axis=1)
acc_original = accuracy_score(preds, labels)
print(f"Accuracy with original data: {round(acc_original*100,1)}%")

Accuracy with original data: 84.6%


In [11]:
model.fit(data, labels)
preds_by_predict = model.predict(data)
acc_pred_by_predict = accuracy_score(preds_by_predict, labels)
print(f"Accuracy with original data: {round(acc_pred_by_predict*100,1)}%")

Accuracy with original data: 98.3%


### 3.3. Finding the class threshold

In [12]:
def compute_class_thresholds(pred_probs: np.ndarray, labels: np.ndarray) -> np.ndarray:
    n_examples, n_classes = pred_probs.shape
    thresholds = np.zeros(n_classes)
    for k in range(n_classes):
        count = 0
        p_sum = 0
        for i in range(n_examples):
            if labels[i] == k:
                count += 1
                p_sum += pred_probs[i, k]
        thresholds[k] = p_sum / count
    return thresholds

<b>Check the data and its label was right</b>

In [13]:
print(dataset_c.loc[230]['Type'])
print("label: " + str(labels.to_numpy()[230]))

1
label: 1


In [14]:
# should be a numpy array of length 5
thresholds = compute_class_thresholds(pred_probs, labels.to_numpy())
thresholds

array([0.69441797, 0.88628718])

### 3.4. Constructing the confident joint

In [15]:
def compute_confident_joint(pred_probs: np.ndarray, labels: np.ndarray, thresholds: np.ndarray) -> np.ndarray:
    n_examples, n_classes = pred_probs.shape
    confident_joint = np.zeros((n_classes, n_classes), dtype=np.int64)
    positions = np.array([[-1, -1]])
    for data_idx in range(n_examples):
        i = labels[data_idx]    #y_noise
        j = None                #y_true -> to find
        #Lưu ý điểm mình bị sai: vị trí của chúng không ứng với label
        p_j = -1
        for candidate_j in range(n_classes):
            p = pred_probs[data_idx, candidate_j]
            if p >= thresholds[candidate_j] and p > p_j:
                j = candidate_j
                p_j = p
        if j is not None:
            confident_joint[i][j] += 1
        positions = np.append(positions, np.array([[i, j]]), axis=0)
    return confident_joint, positions

In [16]:
C, positions = compute_confident_joint(pred_probs, labels.to_numpy(), thresholds)
print(C)

[[206  52]
 [ 38 524]]


### 3.5 Count the number of label issues

In [17]:
num_label_issues = C.sum() - C.trace()
num_label_issues

90

In [18]:
print('Estimated noise rate: {:.1f}%'.format(100*num_label_issues / pred_probs.shape[0]))

Estimated noise rate: 9.1%


### 3.6. Filter out label issues

In [19]:
pred_probs.shape

(988, 2)

In [20]:
self_confidences = []
for i in range(pred_probs.shape[0]):
    self_confidences.append(pred_probs[i, labels[i]])
self_confidences = np.array(self_confidences)

In [21]:
ranked_indices = np.argsort(self_confidences)
ranked_indices[0:5]

array([253, 192, 895, 133, 661])

In [22]:
issue_idx = ranked_indices[:num_label_issues]
print(len(issue_idx))
issue_idx[0]

90


253

In [ ]:
# print_issue_index = pd.DataFrame(issue_idx)
# print_issue_index.to_csv("../output/other/issue_idx.csv")

In [23]:
dataset_c.iloc[ranked_indices[:5]]

,index,ID,SMILES,IC50_mean,Type
253,253,CHEMBL3907792,Nc1ccccc1NC(=O)CCCCNC(=O)c1ccnc(N2CCCCC2)c1,7.455,0
192,192,CHEMBL3896998,Nc1ccc(-c2ccccc2)cc1NC(=O)c1ccc2c(c1)c(C1CC1)c...,1.905,0
895,105,CHEMBL3735106,C[C@]12CC[C@@H]3c4ccc(O)cc4C[C@@H](CCCCCCCCCCC...,0.960,1
133,133,CHEMBL1243261,O=C(CCCCc1cc(C(=O)NO)no1)Nc1ccccc1,26.700,0
661,661,CHEMBL3356525,O=C1CCC[C@@H](C(=O)N[C@H](CCCCCS)C(=O)Nc2ccccc...,17.100,0


# Train with unclean data

In [24]:
unclean_dataset = dataset_c
len(unclean_dataset)

988

In [25]:
unclean_dataset.head()

,index,ID,SMILES,IC50_mean,Type
0,0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.1290,1
1,1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.4600,1
2,2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.0055,1
3,3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.2140,1
4,4,CHEMBL178456,O=C(CS)NCCCCCC(=O)Nc1cccc2cccnc12,1.3200,0


In [26]:
print(len(unclean_dataset.loc[unclean_dataset['Type'] == 0]))
print(len(unclean_dataset.loc[unclean_dataset['Type'] == 1]))

316
672


In [27]:
unclean_labels = unclean_dataset['Type']
unclean_data = maccs_fpts(unclean_dataset.SMILES)
unclean_data = pd.DataFrame(data=unclean_data)

In [28]:
print(unclean_data.shape)
print(unclean_labels.shape)

(988, 167)
(988,)


## Train-test split

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(unclean_data, unclean_labels, test_size=0.2, random_state=1)

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(790, 167)
(198, 167)
(790,)
(198,)


In [31]:
print(len(y_train.loc[y_train == 1]))
print(len(y_train.loc[y_train == 0]))

532
258


In [32]:
print(len(y_test.loc[y_test == 1]))
print(len(y_test.loc[y_test == 0]))

140
58


# Train a More Robust Model without label errors

Now that we have the indices of potential label errors within our data, let's remove them from our data, retrain our model, and see what improvement we can gain.

Let's use a very simple method to handle these label errors and just drop them entirely from the data and retrain our exact same `XGBClassifier`. In a real-world application, a better approach might be to have humans review the issues and _correct_ the labels rather than dropping the data points.

In [43]:
clean_dataset = dataset.drop(issue_idx)
clean_dataset = clean_dataset.reset_index()
len(clean_dataset)

898

In [44]:
clean_dataset.head()

,level_0,index,ID,SMILES,IC50_mean,Type
0,0,0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.12900,1
1,1,1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.46000,1
2,2,2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.00550,1
3,3,3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.21400,1
4,5,5,CHEMBL3601778,Cc1ccc(Nc2nccc3ccc(NCc4ccc(/C=C/C(=O)NO)cc4)cc...,0.17648,1


In [45]:
print(len(clean_dataset.loc[clean_dataset['Type'] == 0]))
print(len(clean_dataset.loc[clean_dataset['Type'] == 1]))

254
644


In [46]:
clean_labels = clean_dataset['Type']
clean_data = maccs_fpts(clean_dataset.SMILES)
clean_data = pd.DataFrame(data=clean_data)

In [47]:
print(clean_data.shape)
print(clean_labels.shape)

(898, 167)
(898,)


# 4. Training with clean data

## 4.1. Train-test split

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(clean_data, clean_labels, test_size=0.2, random_state=1)

In [49]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(718, 167)
(180, 167)
(718,)
(180,)


In [50]:
print(len(y_train.loc[y_train == 1]))
print(len(y_train.loc[y_train == 0]))

520
198


In [51]:
print(len(y_test.loc[y_test == 1]))
print(len(y_test.loc[y_test == 0]))

124
56


## 4.2. Training models

In [52]:
from sklearn.neighbors import KNeighborsClassifier
knn_maccs = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
knn_maccs.fit(X_train, y_train)

KNeighborsClassifier()

In [53]:
from sklearn.ensemble import RandomForestClassifier
rf_maccs = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
rf_maccs.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

In [54]:
from sklearn.svm import SVC
svm_maccs = SVC(kernel='rbf', probability=True, random_state=0)
svm_maccs.fit(X_train, y_train)

SVC(probability=True, random_state=0)

In [55]:
from xgboost import XGBClassifier
bst_maccs = XGBClassifier(n_estimators=100, objective='binary:logistic')
bst_maccs.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## 4.3. Model evaluation

### 4.3.1. Accuracy, Sensitivity, Specificity

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [57]:
X_Total = np.concatenate((X_train, X_test), axis=0)
y_Total = np.concatenate((y_train, y_test), axis=0)

#KNN
cv = KFold(n_splits=10, random_state=1, shuffle=True)
knn_scores = cross_val_score(knn_maccs, X_Total, y_Total, scoring='accuracy', cv=cv, n_jobs=-1)
print('Độ chính xác của 10-fold cross validation KNN: %.3f (%.3f)' % (knn_scores.mean(), knn_scores.std()))

#Random forest
cv = KFold(n_splits=10, random_state=1, shuffle=True)
rf_scores = cross_val_score(rf_maccs, X_Total, y_Total, scoring='accuracy', cv=cv, n_jobs=-1)
print('Độ chính xác của 10-fold cross validation RF: %.3f (%.3f)' % (rf_scores.mean(), rf_scores.std()))

#SVM
cv = KFold(n_splits=10, random_state=1, shuffle=True)
svm_scores = cross_val_score(svm_maccs, X_Total, y_Total, scoring='accuracy', cv=cv, n_jobs=-1)
print('Độ chính xác của 10-fold cross validation SVM: %.3f (%.3f)' % (svm_scores.mean(), svm_scores.std()))

#xg_boost
cv = KFold(n_splits=10, random_state=1, shuffle=True)
bst_scores = cross_val_score(bst_maccs, X_Total, y_Total, scoring='accuracy', cv=cv, n_jobs=-1)
print('Độ chính xác của 10-fold cross validation XG_Boost: %.3f (%.3f)' % (bst_scores.mean(), bst_scores.std()))

Độ chính xác của 10-fold cross validation KNN: 0.906 (0.019)
Độ chính xác của 10-fold cross validation RF: 0.940 (0.024)
Độ chính xác của 10-fold cross validation SVM: 0.894 (0.022)
Độ chính xác của 10-fold cross validation XG_Boost: 0.947 (0.016)


In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score
from tabulate import tabulate
import math

def model_evaluation_calculation(cm):
    tp = cm[0][0]; tn = cm[1][1]; fp = cm[0][1]; fn = cm[1][0]
    ac = (tp+tn)/(tp+tn+fp+fn)
    se = tp/(tp+fn)
    sp = tn/(tn+fp)
    mcc = (tp*tn - fp*fn) / math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return ac, se, sp, mcc

def me_result(cm, model_name):
    cm_string = "Confusion matrix of " + model_name
    print(cm_string)
    print(cm)
    ac, se, sp, mcc = model_evaluation_calculation(cm)
    print("Comparision:")
    table = [[' ' 'AC', 'SE', 'SP', 'MCC'], [model_name, ac, se, sp, mcc]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [59]:
#KNN
y_knn_pred = knn_maccs.predict(X_test)
knn_cm = confusion_matrix(y_test, y_knn_pred)
me_result(knn_cm, model_name="KNN")

#Random Forest
y_rf_pred = rf_maccs.predict(X_test)
rf_cm = confusion_matrix(y_test, y_rf_pred)
me_result(rf_cm, model_name="Random forest")

#SVM
y_svm_pred = svm_maccs.predict(X_test)
svm_cm = confusion_matrix(y_test, y_svm_pred)
me_result(svm_cm, model_name="SVM")

#XG Boost
y_bst_pred = bst_maccs.predict(X_test)
bst_cm = confusion_matrix(y_test, y_bst_pred)
me_result(bst_cm, model_name="XG Boost")

Confusion matrix of KNN
[[ 34  22]
 [  3 121]]
Comparision:
╒═════╤══════════╤══════════╤══════════╤══════════╕
│     │       AC │       SE │       SP │      MCC │
╞═════╪══════════╪══════════╪══════════╪══════════╡
│ KNN │ 0.861111 │ 0.918919 │ 0.846154 │ 0.667831 │
╘═════╧══════════╧══════════╧══════════╧══════════╛
Confusion matrix of Random forest
[[ 44  12]
 [  4 120]]
Comparision:
╒═══════════════╤══════════╤══════════╤══════════╤══════════╕
│               │       AC │       SE │       SP │      MCC │
╞═══════════════╪══════════╪══════════╪══════════╪══════════╡
│ Random forest │ 0.911111 │ 0.916667 │ 0.909091 │ 0.788779 │
╘═══════════════╧══════════╧══════════╧══════════╧══════════╛
Confusion matrix of SVM
[[ 31  25]
 [  5 119]]
Comparision:
╒═════╤══════════╤══════════╤══════════╤══════════╕
│     │       AC │       SE │       SP │      MCC │
╞═════╪══════════╪══════════╪══════════╪══════════╡
│ SVM │ 0.833333 │ 0.861111 │ 0.826389 │ 0.594019 │
╘═════╧══════════╧══════════╧═══

### 4.3.1. AUC

In [60]:
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

knn_y_proba = knn_maccs.predict_proba(X_test)[:, 1]
rf_y_proba = rf_maccs.predict_proba(X_test)[:, 1]
svm_y_proba = svm_maccs.predict_proba(X_test)[:, 1]
bst_y_proba = bst_maccs.predict_proba(X_test)[:, 1]

knn_auc_score = roc_auc_score(y_test, knn_y_proba)
rf_auc_score = roc_auc_score(y_test, rf_y_proba)
svm_auc_score = roc_auc_score(y_test, svm_y_proba)
bst_auc_score = roc_auc_score(y_test, bst_y_proba)
print(knn_auc_score, rf_auc_score, svm_auc_score, bst_auc_score)

0.9225950460829493 0.9660138248847926 0.8951612903225807 0.9733582949308756
